In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import preprocessing
from tensorflow.keras import layers
from keras.layers import LSTM
from keras.layers import concatenate
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
import pandas as pd
df_train=pd.read_csv("./NSL_KDD_Dataset/KDDTrain+.txt")
df_test=pd.read_csv("./NSL_KDD_Dataset/KDDTest+.txt")

In [ ]:
df_train.head()

In [ ]:
df_train.dropna(inplace=True,axis=1) # For now, just drop NA's 
# (rows with missing values)

# The CSV file has no column heads, so add them
df_train.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'attack_type',
    'Score'
]


In [ ]:
df_train.drop('Score',axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.head()

In [ ]:
df_train.groupby('attack_type')['attack_type'].count()

In [ ]:
label_encoder1 = preprocessing.LabelEncoder() 
df_train['protocol_type']= label_encoder1.fit_transform(df_train['protocol_type']) 
a=label_encoder1.classes_ 
label_encoder1.classes_

In [ ]:
label_encoder1 = preprocessing.LabelEncoder() 
df_train['flag']= label_encoder1.fit_transform(df_train['flag']) 
a=label_encoder1.classes_ 
label_encoder1.classes_

In [ ]:
label_encoder1 = preprocessing.LabelEncoder() 
df_train['service']= label_encoder1.fit_transform(df_train['service']) 
a=label_encoder1.classes_ 
label_encoder1.classes_

In [ ]:
label_encoder1 = preprocessing.LabelEncoder() 
df_train['attack_type']= label_encoder1.fit_transform(df_train['attack_type']) 
a=label_encoder1.classes_ 
label_encoder1.classes_

In [ ]:
int_features=['tcp','private','REJ']
int_features

In [ ]:
for i in range(len(a)):
        if a[i]==int_features[0]:
            int_features[0]=i


int_features

In [ ]:
df_train.head()

In [ ]:
y=df_train['attack_type']   #labels
x=df_train.drop(['attack_type'],axis=1) #feature

In [ ]:
x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

In [ ]:
x_train.head()

In [ ]:
y_train.shape

In [ ]:
x_train_1 = x_train[x_train.columns[0:15]]
x_train_2 = x_train[x_train.columns[15:30]]
x_train_3 = x_train[x_train.columns[30:41]]

In [ ]:
x_test_1 = x_test[x_test.columns[0:15]]
x_test_2 = x_test[x_test.columns[15:30]]
x_test_3 = x_test[x_test.columns[30:41]]

In [ ]:
x_train_3.shape

In [ ]:
y_train

In [ ]:
layer_one=Sequential()
layer_one.add(Conv1D(8, 3, activation='relu', input_shape=(15, 1), name = 'Conv1D-1'))
layer_one.add(MaxPooling1D(2, name = 'MaxPool-1'))
layer_one.add(Flatten(name = 'flatten-1'))
layer_one.add(Dense(32, activation='relu', name = 'dense-1'))

layer_two=Sequential()
layer_two.add(Conv1D(8, 3, activation='relu', input_shape=(15, 1), name = 'Conv1D-2'))
layer_two.add(MaxPooling1D(2, name = 'MaxPool-2'))
layer_two.add(Flatten(name = 'flatten-2'))
layer_two.add(Dense(32, activation='relu', name = 'dense-2'))

layer_three=Sequential()
layer_three.add(Conv1D(8, 3, activation='relu', input_shape=(11, 1), name = 'Conv1D-3'))
layer_three.add(MaxPooling1D(2, name = 'MaxPool-3'))
layer_three.add(Flatten(name = 'flatten-3'))
layer_three.add(Dense(32, activation='relu', name = 'dense-3'))

combined=concatenate([layer_one.output, layer_two.output, layer_three.output], name = 'concatenate')
z=Dense(128, activation='relu', name = 'dense-4')(combined)
z=Dense(20, activation='relu', name = 'dense-5')(z)
z=Dropout(0.25, name = 'drop_out')(z)
z=Dense(5, activation='relu', name = 'dense-6')(z)


model=Model(inputs=[layer_one.input, layer_two.input, layer_three.input], outputs=z)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto',
                           restore_best_weights=True)
model.fit([x_train_1,x_train_2,x_train_3],[y_train,y_train,y_train],validation_data=([x_test_1,x_test_2,x_test_3],[y_test,y_test,y_test]),
          callbacks=[monitor],verbose=2,epochs=1000)

In [ ]:
from sklearn.metrics import f1_score
from sklearn import metrics
pred = model.predict([x_test_1,x_test_2,x_test_3])
pred = np.argmax(pred,axis=1)

y_eval = np.argmax([y_test,y_test,y_test],axis=0)
score = metrics.accuracy_score(y_eval, pred)
f1score= f1_score(y_eval, pred, average='micro')
print("Validation score: {}".format(score))
print("f1 score: {}".format(f1score))
print(metrics.confusion_matrix(y_eval, pred))